In [1]:
from IPython.display import display
import re

In [1]:
pip install cameo

Note: you may need to restart the kernel to use updated packages.


In [6]:
from cameo import models
from cameo.strain_design import pathway_prediction

ImportError: cannot import name 'knock_out_model_genes' from 'cobra.manipulation.delete' (c:\Users\pauli\anaconda3\Lib\site-packages\cobra\manipulation\delete.py)

In [2]:
pip install cobra

Note: you may need to restart the kernel to use updated packages.
